# Jet Substructure

Because boosted jets represent the hadronic products of a heavy particle produced with high momentum, some tools have been developed to study the internal structure of these jets. This topic is usually called Jet Substructure. 

Jet substructure algorithms can be divided into three main tools:
 * **grooming algorithms** attempt to reduce the impact of *soft* contributions to clustering sequence by adding some other criteria. Examples of these algorimths are softdrop, trimming, pruning.
 * **subtructure variables** are observables that try to quantify how many cores or prongs can be identify within the structure of the boosted jet. Examples of these variables are n-subjetiness or energy correlation functions.
 * **taggers** are more sofisticated algorithms that attempt to identify the origin of the boosted jet. Currently taggers are based on sofisticated machine-learning techniques which try to use as much information as possible in order to efficiency identify boosted W/Z/Higgs/top jets. Examples of these taggers in CMS are deepAK8/ParticleNet or deepDoubleB.
 
For further reading, several measurements have been performed about jet substructure:
 * [Studies of jet mass in dijet and W/Z+jet events](http://arxiv.org/abs/1303.4811) (CMS).
 * [Jet mass and substructure of inclusive jets in sqrt(s) = 7 TeV pp collisions with the ATLAS experiment](http://arxiv.org/abs/1203.4606) (ATLAS).
 * [Theory slides](http://www.hri.res.in/~sangam/sangam18/talks/Marzani-2.pdf) 
 * [More theory slides]( http://indico.hep.manchester.ac.uk/getFile.py/access?contribId=14&resId=0&materialId=slides&confId=4413)
 * [Talk from Phil Harris](https://web.pa.msu.edu/seminars/hep_seminars/abstracts/2018/Harris-HEPSeminar-Slides-4172018.pdf) on searching for boosted $W$ bosons.
 
 In this part of the tutorial, we will compare different subtructure algorithms as well as some usually subtructure variables.

The code we will use `$CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_make_histograms.py` is a python-based script accessing miniAOD information. Additionally, the code also fills different histograms that we will compare in the next steps of the tutorial. 

The content of the script can be seen [here](https://github.com/cms-jet/JMEDAS/blob/DASSep2020/scripts/jmedas_make_histograms.py). The code will take several minutes to run, so you can launch the script first, and read the script while the code is running. You have probably already looked into this script in the previous exercises. Now execute the following commands in your cmslpc ssh session:

In [ ]:
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_make_histograms.py \
    --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/rsgluon_ttbar_3000GeV.txt \
    --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_ttbar_3000GeV.root \
    --maxevents=2000 \
    --maxFiles 1 \
    --maxjets=6
    #--correctJets Fall17_17Nov2017_V32_MC
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_make_histograms.py \
    --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/ttjets.txt \
    --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/ttjets.root \
    --maxevents=2000 \
    --maxjets=6 \
    --maxFiles 5
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_make_histograms.py \
    --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/WJetsToQQ_HT600to800.txt \
    --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/WJetsToQQ_HT600to800.root \
    --maxevents=2000 \
    --maxjets=4 \
    --maxFiles 2 \
    --matchPdgIdAK8 24 0.8    
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_make_histograms.py \
    --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/QCD_Pt_470to600.txt \
    --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/QCD_Pt_470to600.root \
    --maxevents=2000 \
    --maxjets=4 \
    --maxFiles 2


## Grooming and PU removal algorithms

Now, let's compare the jet masses for ungroomed, pruned, soft drop (SD), PUPPI, and SD+PUPPI:

In [ ]:
import ROOT
%jsroot off

ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetOptTitle(0)
f = ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_ttbar_3000GeV.root")

h_mAK8   = f.Get("h_mAK8")
h_msoftdropAK8 = f.Get("h_msoftdropAK8")
h_mprunedAK8   = f.Get("h_mprunedAK8")
h_mpuppiAK8 = f.Get("h_mpuppiAK8")
h_mSDpuppiAK8 = f.Get("h_mSDpuppiAK8")

h_mAK8.SetLineColor(ROOT.kBlack)
h_mAK8.SetLineWidth(1)
h_msoftdropAK8.SetLineColor(ROOT.kRed-4)
h_msoftdropAK8.SetLineStyle(3)
h_msoftdropAK8.SetLineWidth(1)
h_mprunedAK8.SetLineColor(ROOT.kMagenta-4) 
h_mprunedAK8.SetLineStyle(4) 
h_mpuppiAK8.SetLineColor(ROOT.kGreen)
h_mpuppiAK8.SetLineStyle(3)
h_mpuppiAK8.SetLineWidth(1)
h_mSDpuppiAK8.SetLineColor(ROOT.kAzure)
h_mSDpuppiAK8.SetLineStyle(2)
h_mSDpuppiAK8.SetLineWidth(3)

legend = ROOT.TLegend(0.6, 0.6, 0.88, 0.88)
legend.SetFillColor(0)
legend.SetBorderSize(0)
legend.AddEntry(h_mAK8, "Ungroomed", 'l')
legend.AddEntry(h_msoftdropAK8, "Soft Drop", 'l')
legend.AddEntry(h_mprunedAK8, "Pruned", 'l')
legend.AddEntry(h_mpuppiAK8, "PUPPI", 'l')
legend.AddEntry(h_mSDpuppiAK8, "PUPPI+SD", 'l')

c_mass = ROOT.TCanvas('c_mass', 'c_mass', 800, 600)
#h_mprunedAK8.SetMaximum(500)
h_mprunedAK8.GetXaxis().SetRangeUser(0., 400.)
h_mprunedAK8.DrawNormalized()
#h_mprunedAK8.GetXaxis().SetRangeUser(0, 400)
#h_mprunedAK8.GetYaxis().SetRangeUser(0, 500)
h_msoftdropAK8.DrawNormalized("same") 
h_mAK8.DrawNormalized("same") 
h_mpuppiAK8.DrawNormalized("same")
h_mSDpuppiAK8.DrawNormalized("same")
h_mprunedAK8.DrawNormalized("axis same")
#h_mprunedAK8.SetMaximum(h_msoftdropAK8.GetMaximum()*1.2)

legend.Draw()
c_mass.Draw()


Notice that the previous plot is interactive. Click on it or move your mouse to zoom in or out.

<details>
<summary>
    <font color='blue'>The histogram should look like this:</font>
</summary>
<img src="../files/ex5_rsg_jetmass.png" width=400px/>
</details>
Note that the histogram has two peaks. What do these correspond to? How do the algorithms affect the relative size of the two populations?

# Substructure Variables

Now, let's compare the different subtructure variables between two different samples. Using the histograms that you created in the previous steps, the next cell is just a function to create comparison plots.

In [ ]:
def compareHistogram(variable, processes=["tt", "rsg", "wqq", "qcd"]):
    hist_files = {
        "tt": ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/ttjets.root"), 
        "rsg": ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/rsgluon_ttbar_3000GeV.root"), 
        "wqq": ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/WJetsToQQ_HT600to800.root"), 
        "qcd": ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/QCD_Pt_470to600.root")
    }
    hists = {}
    for iprocess, process in enumerate(processes):
        #print(process)
        hists[process] = hist_files[process].Get("h_{}".format(variable))
        if not hists[process]:
            print "ERROR : didn't find histogram {} in file {}".format("h_{}".format(variable), hist_files[process].GetPath())
        hists[process].SetDirectory(0)
        hists[process].SetLineColor(iprocess+1)
        hists[process].Rebin(2)
        if hists[process].Integral() > 0:
            hists[process].Scale(1. / hists[process].Integral())
    
    # Dummy histogram for drawing axes on TCanvas
    #hframe = ROOT.TH1F("hframe", "hframe", hists["tt"].GetNbinsX(), hists["tt"].GetXaxis().GetXmin(), hists["tt"].GetXaxis().GetXmax())
    #hframe.SetDirectory(0)
    #hframe.GetXaxis().SetTitle(variable)
    #hframe.SetMinimum(0.)
    #hframe.SetMaximum(1.2 * max([hists[process].GetMaximum() for process in ["tt", "rsg", "wqq"]]))

    if variable == "ak8_N2_beta1":
        hists[processes[0]].GetXaxis().SetRangeUser(0., 0.5)
    elif variable == "logrhoRatioAK8":
        for process in processes:
            hists[process].Rebin(5)

    hists[processes[0]].SetMaximum(1.2 * max([hists[process].GetMaximum() for process in processes]))

    legend = ROOT.TLegend(0.15, 0.65, 0.4, 0.85)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    
    legend_entries = {
        "tt": "t#bar{t}", 
        "rsg": "RS KK gluon",
        "wqq": "W(qq)", 
        "qcd": "QCD"
    }
    
    for process in processes:
        legend.AddEntry(hists[process], legend_entries[process], "l")
    
    canvas = ROOT.TCanvas(variable, variable, 700, 500) 

    #hframe.Draw()
    for iprocess, process in enumerate(processes):
        if iprocess == 0:
            draw_opts = "hist"
        else:
            draw_opts = "hist same"
        hists[process].Draw(draw_opts)
    
    legend.Draw()
    canvas.Draw()
    return canvas, legend, hists


Let's start with n-subjetiness ratios. The variable $\tau_N$ gives a sense of how many N prongs or cores can be find inside the jet. It is known that the n-subjetiness variables itself ($\tau_{N}$) do not provide good discrimination power, but its ratios do. Then, a $\tau_{MN} = \dfrac{\tau_M}{\tau_N}$ basically tests if the jet is more M-prong compared to N-prong. For instance, we expect 2 prongs for boosted jets originated from hadronic Ws, while we expect 1 prongs for high-pt jets from QCD multijet processes.

Let's compare one of the most common nsubjetiness ratio $\tau_{21}$:

In [ ]:
compareHistogram('tau21AK8', processes=["rsg", "wqq", "qcd"])

What can you say about the two histograms? Is $\tau_{21}$ telling you something about the nature of the boosted jets selected?

Let's compare now $\tau_{32}$:

In [ ]:
compareHistogram('tau32AK8', processes=["rsg", "qcd"])

In [ ]:
compareHistogram('tau32AK8_pt450', processes=["rsg", "qcd"])

What can you say about the two histograms? Is $\tau_{32}$ telling you something about the nature of the boosted jets selected?

Another subtructure variable commonly used is the energy correlation function $N2$. Similarly than $\tau_{21}$, $N2$ tests if the boosted jet is compatible with a 2-prong jet hypothesis. Let's compare now $N2$ and $N3$:

In [ ]:
compareHistogram( 'ak8_N2_beta1', processes=["rsg", "wqq", "qcd"] )

In [ ]:
compareHistogram( 'ak8_N3_beta1_pt450', processes=["rsg", "qcd"] )

What can you say about the two histograms? Are $N2$ and $N3$ telling you something about the nature of the boosted jets selected?

# $\rho$ parameter
A useful variable for massive, fat jets is the QCD scaling parameter $\rho$, defined as:

$\rho=\log(m^2/(p_{\mathrm{T}}R)^2)$.

(Sometimes $\rho$ is defined without the log). One useful feature of this variable is that QCD jet mass grows with $p_{\mathrm{T}}$, i.e. the two quantities are strongly correlated, while $\rho$ is much less correlated with $p_{\mathrm{T}}$.



In [ ]:
compareHistogram( 'logrhoRatioAK8' )
#compareHistogram( 'rhoRatioAK8' )

In which cases do you think the $\rho$ variable can be used? 

<details>
<summary>
    <font color='blue'>Click to see more.</font>
</summary>
The following two plots show what QCD events look like in different $p_{T}$ ranges. It's clear that the mass depends very strongly on $p_{T}$, while the $\rho$ shape is fairly constant vs. $p_{T}$ (ignoring $\rho<7$ or so, which is the non-perturbative region). Having a stable shape is useful when studying QCD across a wide $p_{T}$ range.
<img src="../files/qcdpt_mass.png" width=600px/>
<img src="../files/qcdpt_rho.png" width=600px/>
</details>
